In [2]:
# # Mounting GCS to colab
# # https://stackoverflow.com/questions/51715268/how-to-import-data-from-google-cloud-storage-to-google-colab

# from google.colab import auth
# auth.authenticate_user()

In [5]:
# !echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
# !curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
# !apt -qq update
# !apt -qq install gcsfuse

In [6]:
# !mkdir colab_indoor
# !gcsfuse indoor-data colab_indoor
# # !mkdir colab_indoor/train_4
# # !gcsfuse indoor-data/train_4 colab_indoor/train_4
# # !mkdir colab_indoor/test_4
# # !gcsfuse indoor-data/test_4 colab_indoor/test_4

In [7]:
!ls -la -h ./colab_indoor/train_4_colcut

total 1.2G
-rw-r--r-- 1 root root  43M Apr 14 21:07 5a0546857ecc773753327266_train.csv
-rw-r--r-- 1 root root  46M Apr 14 21:07 5c3c44b80379370013e0fd2b_train.csv
-rw-r--r-- 1 root root 113M Apr 14 21:08 5d27075f03f801723c2e360f_train.csv
-rw-r--r-- 1 root root  43M Apr 14 21:08 5d27096c03f801723c31e5e0_train.csv
-rw-r--r-- 1 root root  50M Apr 14 21:08 5d27097f03f801723c320d97_train.csv
-rw-r--r-- 1 root root  12M Apr 14 21:08 5d27099f03f801723c32511d_train.csv
-rw-r--r-- 1 root root  17M Apr 14 21:08 5d2709a003f801723c3251bf_train.csv
-rw-r--r-- 1 root root  72M Apr 14 21:09 5d2709b303f801723c327472_train.csv
-rw-r--r-- 1 root root  81M Apr 14 21:09 5d2709bb03f801723c32852c_train.csv
-rw-r--r-- 1 root root  48M Apr 14 21:09 5d2709c303f801723c3299ee_train.csv
-rw-r--r-- 1 root root  48M Apr 14 21:09 5d2709d403f801723c32bd39_train.csv
-rw-r--r-- 1 root root  51M Apr 14 21:09 5d2709e003f801723c32d896_train.csv
-rw-r--r-- 1 root root 3.9M Apr 14 21:10 5da138274db8ce0c98bbd3d2_train.csv
-

---
## DLSTM Implementation
---

In [64]:
import random
from random import sample
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
from scipy.ndimage.filters import uniform_filter1d
from scipy.interpolate import interp1d
import time
from collections import defaultdict
import seaborn as sns

import scipy.stats as stats
from pathlib import Path
import glob
import pickle
from tqdm import tqdm
import random
import os
import copy
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import multiprocessing

EPOCH = 50 # default at 50
BATCH_SIZE = 4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
FOLDS = 5

NUM_CORES = multiprocessing.cpu_count()
print(NUM_CORES)

OUTPUT_NAME = "train_8_colcut_DLSTM"

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

set_seed()

2


In [9]:
# train paths and test paths
train_files = sorted(glob.glob("./colab_indoor/train_4_colcut/*"))
test_files = sorted(glob.glob("./colab_indoor/test_4_colcut/*"))

# load submission file
sub_df = pd.read_csv("./colab_indoor/sample_submission.csv", index_col=0)
# sub_df[["site", "file", "timestamp"]] = sub_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
display(sub_df.head())

,floor,x,y
site_path_timestamp,,,
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000000009,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000009017,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000015326,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000018763,0,75.0,75.0
5a0546857ecc773753327266_046cfa46be49fc10834815c6_0000000022328,0,75.0,75.0


In [10]:
# Load train csv and test csv
train_df = pd.read_csv(train_files[0])
test_df = pd.read_csv(test_files[0])
display(train_df.head())
display(test_df.head())

,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,a52823c2ed57e18f81da316e5bcac8bd2754ce96,61a14256d195624aadd9dfd55c8643505635edd7,c93c29d2173b811a18de34940ccc210a3064230e,a09ab3d8a7700fec7b83389c06088c91748be41c,4cff1c8cfec27801ddc9a690ade87c57f1142ee0,c59f0226dcdf235d7cfaba662bbdb5d31b4c3f14,5b225e187d0dec3110683a74d0c9a5a4cb2022f5,d84cce12fbfba61bf930123050f61a11e2a29310,b2546cae6e588d38618eacc557dd0385812197cf,b26914599f6d9ba16b43975394e1eeb9d82f4bab,68616cd56e6c03f67852c7394ddde68b07d7a3c6,c703b0a9624cbb37e30e71f4dcfe9347013fcbde,b0b1a58ed86926fdd2a0c584a85834d1cc9d2e0c,d6fde84d77e0618496709be2037bc47d952d69ce,ca0e0bebf85f1348808b0979b74868e4e675e4d8,6bc91b3951089c3a225396608b138ca178479924,f65804069553e141a96eae17f475584617a5324b,77d933b398825a1c6a9054bde71d44d2d3cb17f7,1aebe03ff86398cd62ed5ec0da3650e9041baa31,5cfff2a06e481233f2df41ef8adc590a0d779517,288f3c61d2f10cc122ff9cd09c27c5aed544e4e2,fb1ebbb8a61b4299bd83248e745335974fdfff59,0c235b02dd008286ea5053621c7ce50df7f17e10,86bd6179355aaeb54bcd541040f23808d6656442,2ad2da7d639746b33d7a6118afad5a2964b58ce5,9cc80ca7212ff8df8aa509760e6759bcb7015da8,c70e999e91f2381a2856197df91987412d6e049e,32ad7772f626e6d531c1de47806b68af6f33c339,b58b9ee349cc63d5c92f6b1c834cdd75cc0392ab,e95b2b7db211a0b001ef5ef18ffbd08cbc4f4e09,9780331db07b369d8a009eb89552004b88334dae,d0f0c9fa52edde81962135fcf5e5b11e53ddd252,...,c08ad78a45798cfe176a42b35c7381ae602711c5,d32dd11040b254cd889c9ead2d4a50f6e3900196,5f2c7898f4ab6a64a6a0d1d0455c48baa2ae89cf,5ee75f958530d980fa6c9a7ff53d9a0967975c00,f745e26c10cd9356249eaf66bd2b30b4ca6fedf1,ee9118a7c5bb4f5cf727ca4f2bf2559e4fa73cca,d237af29bda22db4e53cda910eb715354c67e5f9,488c67b452c1e27790b21aae09b920692a461c94,e32d9e78dd87486b832f7723c916ca93ad686a54,40469d563203172ce5026dcf8fbd01c2a382c315,17e9597a21e9085d85d0e3eeb857528a6067ea5c,87b73e43d586ac602a5ae8834e047e605599a2d1,dc3b0421516fc5fd03368e35683437dfc4730fcb,207781229d24ce81fcf1acf4edb187e9098ffeb4,dc522df53e28d76a729dc56f81b1295a50ce4289,9b5fa221ca70b4112140a05be4aca04fcc6a6284,4b82b58b90f42cf69fb1258b8215584b8b326004,9baa43ea53b17b875a5e9d7b346eb7a17a147837,ac3f58143d5c21d3a4203d0d6311c7f1e81d72a1,15c9df1d9c38509443a2d4d39ca80ce0cc42ea1c,b17a526713611a0516cadcbbb6f7890643475119,d30db6f81498ea2f1b5b1496ef47da7c97dd222a,41d5e67c977d3d040538d2829084484d2c02302b,ddebbf6314c91ed820c9c64dcc6f38dcb50ce59a,bccd6a9054f8649ad43fe96b766687fb769b064f,82e5ed37cfa2d5c2a75ec0668174886f33f25434,60cb877272c804a7be6af18a33c4a9e744eb706d,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,c06252f47ae31b9bf221588e76f7f929f98fd37f,3175ae1d61c8d7c247d006bedd09a52ed70a9538,ccb72d20e32993a8015e6886612d8d56e6d393c0,509d1f842b0773e85c6beec0bb530542efd35cb9,9d404663da83e8e1c90c8d33c5ad27b4e0763bf5,56b206b82049935c495aa67687652b18f9db9ea4,975175309f52a0424020db06c9600218c46f5f74,f72e76e1198e03e8747cbab7d5eb7c8892236538,baea70810408b4f206f7487de8976e8ea8ccd889,06a0c0f87f97918d1642db21285d7182dc1b7d15,fd0bdf5a4dca2566935b14a78c441846b4fbda57,83eb995009dcfb8579702ca9ce81e2f358e2336f
0,1578470975068,1913,141.92305,82.461280,B1,-1,5e158ecff4c3420006d52164,5a0546857ecc773753327266,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,1578470976967,3812,141.92305,82.461280,B1,-1,5e158ecff4c3420006d52164,5a0546857ecc773753327266,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,1578470978867,3120,142.00758,87.245865,B1,-1,5e158ecff4c3420006d52

,site_path_timestamp,correct_wps_ts,wifi_ts,wps_diff,x,y,floor,floor_int,file_id,site_id,a52823c2ed57e18f81da316e5bcac8bd2754ce96,61a14256d195624aadd9dfd55c8643505635edd7,c93c29d2173b811a18de34940ccc210a3064230e,a09ab3d8a7700fec7b83389c06088c91748be41c,4cff1c8cfec27801ddc9a690ade87c57f1142ee0,c59f0226dcdf235d7cfaba662bbdb5d31b4c3f14,5b225e187d0dec3110683a74d0c9a5a4cb2022f5,d84cce12fbfba61bf930123050f61a11e2a29310,b2546cae6e588d38618eacc557dd0385812197cf,b26914599f6d9ba16b43975394e1eeb9d82f4bab,68616cd56e6c03f67852c7394ddde68b07d7a3c6,c703b0a9624cbb37e30e71f4dcfe9347013fcbde,b0b1a58ed86926fdd2a0c584a85834d1cc9d2e0c,d6fde84d77e0618496709be2037bc47d952d69ce,ca0e0bebf85f1348808b0979b74868e4e675e4d8,6bc91b3951089c3a225396608b138ca178479924,f65804069553e141a96eae17f475584617a5324b,77d933b398825a1c6a9054bde71d44d2d3cb17f7,1aebe03ff86398cd62ed5ec0da3650e9041baa31,5cfff2a06e481233f2df41ef8adc590a0d779517,288f3c61d2f10cc122ff9cd09c27c5aed544e4e2,fb1ebbb8a61b4299bd83248e745335974fdfff59,0c235b02dd008286ea5053621c7ce50df7f17e10,86bd6179355aaeb54bcd541040f23808d6656442,2ad2da7d639746b33d7a6118afad5a2964b58ce5,9cc80ca7212ff8df8aa509760e6759bcb7015da8,c70e999e91f2381a2856197df91987412d6e049e,32ad7772f626e6d531c1de47806b68af6f33c339,b58b9ee349cc63d5c92f6b1c834cdd75cc0392ab,e95b2b7db211a0b001ef5ef18ffbd08cbc4f4e09,...,c08ad78a45798cfe176a42b35c7381ae602711c5,d32dd11040b254cd889c9ead2d4a50f6e3900196,5f2c7898f4ab6a64a6a0d1d0455c48baa2ae89cf,5ee75f958530d980fa6c9a7ff53d9a0967975c00,f745e26c10cd9356249eaf66bd2b30b4ca6fedf1,ee9118a7c5bb4f5cf727ca4f2bf2559e4fa73cca,d237af29bda22db4e53cda910eb715354c67e5f9,488c67b452c1e27790b21aae09b920692a461c94,e32d9e78dd87486b832f7723c916ca93ad686a54,40469d563203172ce5026dcf8fbd01c2a382c315,17e9597a21e9085d85d0e3eeb857528a6067ea5c,87b73e43d586ac602a5ae8834e047e605599a2d1,dc3b0421516fc5fd03368e35683437dfc4730fcb,207781229d24ce81fcf1acf4edb187e9098ffeb4,dc522df53e28d76a729dc56f81b1295a50ce4289,9b5fa221ca70b4112140a05be4aca04fcc6a6284,4b82b58b90f42cf69fb1258b8215584b8b326004,9baa43ea53b17b875a5e9d7b346eb7a17a147837,ac3f58143d5c21d3a4203d0d6311c7f1e81d72a1,15c9df1d9c38509443a2d4d39ca80ce0cc42ea1c,b17a526713611a0516cadcbbb6f7890643475119,d30db6f81498ea2f1b5b1496ef47da7c97dd222a,41d5e67c977d3d040538d2829084484d2c02302b,ddebbf6314c91ed820c9c64dcc6f38dcb50ce59a,bccd6a9054f8649ad43fe96b766687fb769b064f,82e5ed37cfa2d5c2a75ec0668174886f33f25434,60cb877272c804a7be6af18a33c4a9e744eb706d,01506a39edc2da8d9003f0f12ce5dcc2e8c2772e,c06252f47ae31b9bf221588e76f7f929f98fd37f,3175ae1d61c8d7c247d006bedd09a52ed70a9538,ccb72d20e32993a8015e6886612d8d56e6d393c0,509d1f842b0773e85c6beec0bb530542efd35cb9,9d404663da83e8e1c90c8d33c5ad27b4e0763bf5,56b206b82049935c495aa67687652b18f9db9ea4,975175309f52a0424020db06c9600218c46f5f74,f72e76e1198e03e8747cbab7d5eb7c8892236538,baea70810408b4f206f7487de8976e8ea8ccd889,06a0c0f87f97918d1642db21285d7182dc1b7d15,fd0bdf5a4dca2566935b14a78c441846b4fbda57,83eb995009dcfb8579702ca9ce81e2f358e2336f
0,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474564146,1578474566477,2331,-999.0,-999.0,-999.0,-999.0,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-69,-70,-68,-69,-62,-68,-48,-48,-48,-47,-63,-64,-66,-65,-65,-48,-60,-64,-65,-63,-66,-65,-58,-61,-57,-58,-61,-60,-57,-58,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-63,-999,-72,-999,-999,-999,-75,-999,-999,-999,-999,-999,-999,-71,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-60
1,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474573154,1578474573645,491,-999.0,-999.0,-999.0,-999.0,046cfa46be49fc10834815c6,5a0546857ecc773753327266,-71,-72,-71,-71,-61,-71,-47,-46,-46,-47,-73,-73,-61,-61,-61,-47,-69,-72,-61,-72,-73,-61,-62,-62,-62,-62,-64,-62,-63,-59,...,-999,-999,-999,-999,-999,-999,-76,-999,-999,-999,-999,-63,-999,-56,-999,-999,-999,-67,-999,-999,-999,-999,-999,-999,-58,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-75
2,5a0546857ecc773753327266_046cfa46be49fc1083481...,1578474579463,157847457

In [81]:
# Match train and test columns
all_train_cols = list(train_df.columns)
all_test_cols = list(test_df.columns)
print("all train cols: ", len(all_train_cols), "\n", "all test cols: ", len(all_test_cols))

# get all non-overlapping columns
no_overlap_col = list(set(all_train_cols) ^ set(all_test_cols))
no_overlap_col += ["floor", "file_id", "site_id"] # add other columns to exclude
train_cols = [x for x in all_train_cols if x not in no_overlap_col]
test_cols = [x for x in all_test_cols if x not in no_overlap_col]
# test_cols += ["site_path_timestamp"] # test_df needs to keep "site_path_timestamp"

# filter out the df by the columns to leave
train_df = train_df[train_cols]
test_df = test_df[test_cols]

# # Drop some columns not necessary as a feature
# drop_cols = ["wifi_ts", "floor", "file_id", "site_id"]
# for df in [train_df, test_df]:
#     df = df.drop(columns=drop_cols, inplace=True)

# Convert df object columns to integers and then the whole thing to tensors
for df in [train_df, test_df]:
    obj_col = list(df.select_dtypes(include=['object']).columns)
    for col in obj_col:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].values)

# # Try without using scalers for now. 
# # Apply MinMaxScaler to each IMU columns
# current_cols = list(train_df.columns)
# imu_cols = current_cols[:23]
# exception_columns = ["wps_diff", "x", "y", "floor_int", "rel_diff", "rel_x", "rel_y"]
# imu_cols = [x for x in imu_cols if x not in exception_columns]
# print("imu_cols: ", imu_cols)
# for col in imu_cols:
#     ss_scaler = StandardScaler()
#     train_df[col] = ss_scaler.fit_transform(train_df[col].values.reshape(-1, 1))
#     test_df[col] = ss_scaler.transform(test_df[col].values.reshape(-1, 1))

print(len(train_df.columns))
print(len(test_df.columns))
print(len(train_df))
print(len(test_df))
print("object dtype columns in train", train_df.select_dtypes(include=['object']).columns)
print("object dtype columns in test", test_df.select_dtypes(include=['object']).columns)
display(train_df.head())
display(test_df.head())

all train cols:  200 
 all test cols:  200
200
200
9296
300
object dtype columns in train Index([], dtype='object')
object dtype columns in test Index([], dtype='object')


,wifi_ts,wps_diff,x,y,floor_int,a52823c2ed57e18f81da316e5bcac8bd2754ce96,61a14256d195624aadd9dfd55c8643505635edd7,c93c29d2173b811a18de34940ccc210a3064230e,a09ab3d8a7700fec7b83389c06088c91748be41c,4cff1c8cfec27801ddc9a690ade87c57f1142ee0,c59f0226dcdf235d7cfaba662bbdb5d31b4c3f14,5b225e187d0dec3110683a74d0c9a5a4cb2022f5,d84cce12fbfba61bf930123050f61a11e2a29310,b2546cae6e588d38618eacc557dd0385812197cf,b26914599f6d9ba16b43975394e1eeb9d82f4bab,68616cd56e6c03f67852c7394ddde68b07d7a3c6,c703b0a9624cbb37e30e71f4dcfe9347013fcbde,b0b1a58ed86926fdd2a0c584a85834d1cc9d2e0c,d6fde84d77e0618496709be2037bc47d952d69ce,ca0e0bebf85f1348808b0979b74868e4e675e4d8,6bc91b3951089c3a225396608b138ca178479924,f65804069553e141a96eae17f475584617a5324b,77d933b398825a1c6a9054bde71d44d2d3cb17f7,1aebe03ff86398cd62ed5ec0da3650e9041baa31,5cfff2a06e481233f2df41ef8adc590a0d779517,288f3c61d2f10cc122ff9cd09c27c5aed544e4e2,fb1ebbb8a61b4299bd83248e745335974fdfff59,0c235b02dd008286ea5053621c7ce50df7f17e10,86bd6179355aaeb54bcd541040f23808d6656442,2ad2da7d639746b33d7a6118afad5a2964b58ce5,9cc80ca7212ff8df8aa509760e6759bcb7015da8,c70e999e91f2381a2856197df91987412d6e049e,32ad7772f626e6d531c1de47806b68af6f33c339,b58b9ee349cc63d5c92f6b1c834cdd75cc0392ab,e95b2b7db211a0b001ef5ef18ffbd08cbc4f4e09,9780331db07b369d8a009eb89552004b88334dae,d0f0c9fa52edde81962135fcf5e5b11e53ddd252,a9535d6a53bc660bdc3f2540251e6289b8c75bfd,2798955a957d1c8c01f6341133cd847016b2b742,5a1b76adb81b4e42fe9955ebfa04cc331a0b2927,...,6ee1dc249422a7d7a47987eea4188c5f3fdedb76,6e102b9ee61aa8028fe0a95e87f3dc386010e0e2,fdf37fa13679f581bdfaae3b99e368633e0a144b,e11207e4d38391623e8a73461dea4612b6d348c2,3419af56cd3a9290bcf16c888777961741c8ec28,19787b1ee33ac886f9784105b5da6548d838195c,63d0d455f48e2be51297f1e695fadde7c49a7025,f0386c68fadad2c008c746e2acfaf6bdb7bc0fd9,0e2250a84319bd6fb557786aa36b3c329f9b4fd1,993675a9a48d090babd1a33fbee5f088ef7c73dd,dd65bd54b10548c410dedc4839baf4f99ed4e8f8,b6d159940a3525d560d1aa0581293960c0766dc2,8eb7c9848aa1e78b0da3b6f9f4f730885b0cd4f0,31fb3a7ff1d2cbbb58e8b8e28a3833ed13186c20,b9432300de5aa9b5582a1d4603e541819d4991ce,6729afdb470f55f886bfbab90cbfa0830a2bb75f,c5a0dc2fd03e99e26963fcd5918c398577e50669,f0eb8e099777a084f99eabefe449833b2bd25876,2caf39807897e4577f271ec67e5701eb36101b2c,76be4331334efa1ab2038e98b71458f6168fdaba,ce3243f4a093f9959b5b377bcfb465c507bb7940,fdc19f011587b75c11a6c30d8ca06d90107b6bde,4074d3eca583baf4fc03a0c0d1d6ae3f6c18f448,60f6d22eb3deec6dab7a568e0e0486c09cdd6a7b,c138892cffe41574f10daf3e50662b9b87c45b78,60ba7403a1a1c27c1d28780bbd48980c16cc894d,3ce53e146c68925e9e5c8871d5f453f35b016c69,e0842697e2c12ca915788d2ffb7045422a044125,707a33db19f2064df5f4e9f03dbb439aff81fbea,524632cb8ad6d175f10c530411e02950474d5cc5,230b0585ecc40e716b7327b977d7f1d357558bf7,680ccefeb88f88e82c6d3b277490a4390d5dbcda,a95ab29a85dd9d8dd4b8915af56a184b2da8a66f,5dfed06a2dc471b7c159ce9176da2ffe187fa478,f3e26cef2e2eefbcca02f9ca07bca97d900b0d3d,f2eb513d4c87747d0e4bc8c3bbf12951b3819678,6cd94cc5251afe872b089db662b5da88bd482af5,2b8287e26eaa418c6bcbf570d4b45ce939b57713,39648d81b21198da0f317efade93c2e7cff7f45f,2316d69de2343b00b2ca25e28e2a7b2224bfaf9d
0,1578470975068,1913,141.92305,82.461280,-1,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
1,1578470976967,3812,141.92305,82.461280,-1,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,...,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999,-999
2,1578470978867,3120,142.00758,87.245865,-1

,wifi_ts,wps_diff,x,y,floor_int,a52823c2ed57e18f81da316e5bcac8bd2754ce96,61a14256d195624aadd9dfd55c8643505635edd7,c93c29d2173b811a18de34940ccc210a3064230e,a09ab3d8a7700fec7b83389c06088c91748be41c,4cff1c8cfec27801ddc9a690ade87c57f1142ee0,c59f0226dcdf235d7cfaba662bbdb5d31b4c3f14,5b225e187d0dec3110683a74d0c9a5a4cb2022f5,d84cce12fbfba61bf930123050f61a11e2a29310,b2546cae6e588d38618eacc557dd0385812197cf,b26914599f6d9ba16b43975394e1eeb9d82f4bab,68616cd56e6c03f67852c7394ddde68b07d7a3c6,c703b0a9624cbb37e30e71f4dcfe9347013fcbde,b0b1a58ed86926fdd2a0c584a85834d1cc9d2e0c,d6fde84d77e0618496709be2037bc47d952d69ce,ca0e0bebf85f1348808b0979b74868e4e675e4d8,6bc91b3951089c3a225396608b138ca178479924,f65804069553e141a96eae17f475584617a5324b,77d933b398825a1c6a9054bde71d44d2d3cb17f7,1aebe03ff86398cd62ed5ec0da3650e9041baa31,5cfff2a06e481233f2df41ef8adc590a0d779517,288f3c61d2f10cc122ff9cd09c27c5aed544e4e2,fb1ebbb8a61b4299bd83248e745335974fdfff59,0c235b02dd008286ea5053621c7ce50df7f17e10,86bd6179355aaeb54bcd541040f23808d6656442,2ad2da7d639746b33d7a6118afad5a2964b58ce5,9cc80ca7212ff8df8aa509760e6759bcb7015da8,c70e999e91f2381a2856197df91987412d6e049e,32ad7772f626e6d531c1de47806b68af6f33c339,b58b9ee349cc63d5c92f6b1c834cdd75cc0392ab,e95b2b7db211a0b001ef5ef18ffbd08cbc4f4e09,9780331db07b369d8a009eb89552004b88334dae,d0f0c9fa52edde81962135fcf5e5b11e53ddd252,a9535d6a53bc660bdc3f2540251e6289b8c75bfd,2798955a957d1c8c01f6341133cd847016b2b742,5a1b76adb81b4e42fe9955ebfa04cc331a0b2927,...,6ee1dc249422a7d7a47987eea4188c5f3fdedb76,6e102b9ee61aa8028fe0a95e87f3dc386010e0e2,fdf37fa13679f581bdfaae3b99e368633e0a144b,e11207e4d38391623e8a73461dea4612b6d348c2,3419af56cd3a9290bcf16c888777961741c8ec28,19787b1ee33ac886f9784105b5da6548d838195c,63d0d455f48e2be51297f1e695fadde7c49a7025,f0386c68fadad2c008c746e2acfaf6bdb7bc0fd9,0e2250a84319bd6fb557786aa36b3c329f9b4fd1,993675a9a48d090babd1a33fbee5f088ef7c73dd,dd65bd54b10548c410dedc4839baf4f99ed4e8f8,b6d159940a3525d560d1aa0581293960c0766dc2,8eb7c9848aa1e78b0da3b6f9f4f730885b0cd4f0,31fb3a7ff1d2cbbb58e8b8e28a3833ed13186c20,b9432300de5aa9b5582a1d4603e541819d4991ce,6729afdb470f55f886bfbab90cbfa0830a2bb75f,c5a0dc2fd03e99e26963fcd5918c398577e50669,f0eb8e099777a084f99eabefe449833b2bd25876,2caf39807897e4577f271ec67e5701eb36101b2c,76be4331334efa1ab2038e98b71458f6168fdaba,ce3243f4a093f9959b5b377bcfb465c507bb7940,fdc19f011587b75c11a6c30d8ca06d90107b6bde,4074d3eca583baf4fc03a0c0d1d6ae3f6c18f448,60f6d22eb3deec6dab7a568e0e0486c09cdd6a7b,c138892cffe41574f10daf3e50662b9b87c45b78,60ba7403a1a1c27c1d28780bbd48980c16cc894d,3ce53e146c68925e9e5c8871d5f453f35b016c69,e0842697e2c12ca915788d2ffb7045422a044125,707a33db19f2064df5f4e9f03dbb439aff81fbea,524632cb8ad6d175f10c530411e02950474d5cc5,230b0585ecc40e716b7327b977d7f1d357558bf7,680ccefeb88f88e82c6d3b277490a4390d5dbcda,a95ab29a85dd9d8dd4b8915af56a184b2da8a66f,5dfed06a2dc471b7c159ce9176da2ffe187fa478,f3e26cef2e2eefbcca02f9ca07bca97d900b0d3d,f2eb513d4c87747d0e4bc8c3bbf12951b3819678,6cd94cc5251afe872b089db662b5da88bd482af5,2b8287e26eaa418c6bcbf570d4b45ce939b57713,39648d81b21198da0f317efade93c2e7cff7f45f,2316d69de2343b00b2ca25e28e2a7b2224bfaf9d
0,1578474566477,2331,-999.0,-999.0,-999.0,-69,-70,-68,-69,-62,-68,-48,-48,-48,-47,-63,-64,-66,-65,-65,-48,-60,-64,-65,-63,-66,-65,-58,-61,-57,-58,-61,-60,-57,-58,-60,-60,-60,-60,-76,...,-74,-74,-78,-65,-82,-74,-61,-69,-73,-85,-76,-77,-65,-82,-999,-82,-83,-80,-999,-74,-999,-83,-86,-74,-999,-83,-75,-83,-74,-83,-70,-70,-69,-84,-73,-51,-83,-73,-69,-82
1,1578474573645,491,-999.0,-999.0,-999.0,-71,-72,-71,-71,-61,-71,-47,-46,-46,-47,-73,-73,-61,-61,-61,-47,-69,-72,-61,-72,-73,-61,-62,-62,-62,-62,-64,-62,-63,-59,-60,-60,-60,-60,-81,...,-67,-77,-75,-58,-84,-76,-67,-74,-66,-88,-88,-76,-66,-85,-999,-84,-69,-80,-67,-75,-999,-86,-87,-77,-999,-70,-77,-79,-68,-83,-73,-65,-70,-84,-73,-69,-82,-82,-69,-86
2,1578474578851,612,-999.0,-999.0,-999.0,-65,-66,-66,-66,-70,-65,-42,-42,-42,-41,-69,-67,-71,-70,-70,-41,-70,-68,-69,-69,-69,-70,-56,-70,-55,-55,-71,-70,-55,-59,-57,-57,-57,-56,-81,...,-75,-8

In [ ]:
# # Use when we need to consider timetamps

# # get timestamp and sort by time
# test_df[["site", "file", "timestamp"]] = test_df["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
# test_df = test_df.drop(columns=["site_path_timestamp", "site", "file"])
# test_df["timestamp"] = test_df["timestamp"].astype(int)
# # display(test_df.head())

# # sort by time
# train_df = train_df.sort_values(by=["file_id", "wifi_ts"])
# test_df = test_df.sort_values(by=["file_id", "timestamp"])
# # display(train_df.head(20))
# # display(test_df.head(20))
# # print(len(test_df.columns))

In [82]:
class IndoorDataset(Dataset):
    def __init__(self, data, flag="train"):
        self.flag = flag
        self.data = data

        all_cols = list(data.columns)
        target_cols = ["x", "y", "floor_int"]
        non_target_cols = [col for col in all_cols if col not in target_cols]
        self.features = data[non_target_cols]

        if self.flag == "train":
            self.x = data.loc[:, "x"]
            self.y = data.loc[:, "y"]
            # self.f = data.loc[:, "floor_int"]

    def __len__(self):
        return len(self.features)

    def feat_width(self):
        return self.features.shape[1]

    def __getitem__(self, index):
        features = self.features.iloc[index, :]
        features_out = torch.tensor(features.to_numpy())
        if self.flag == "train":
            x = self.x[index]
            y = self.y[index]
            # f = self.f[index]
            x_out = torch.tensor(x)
            y_out = torch.tensor(y)
            # f_out = torch.tensor(f)
            return features_out, x_out, y_out
        else:
            return features_out

In [83]:
# Create train and test Dataset
train_ds = IndoorDataset(train_df)
test_ds = IndoorDataset(test_df, flag="test")

one_train_ds = train_ds.__getitem__(1000)
print("train ds len: ", train_ds.__len__())
print("train ds features: ", one_train_ds[0])
print("train ds x: ", one_train_ds[1])
print("train ds y: ", one_train_ds[2])
# print("train ds f: ", one_train_ds[3])

one_test_ds = test_ds.__getitem__(0)
print("test ds len: ", test_ds.__len__())
print("test ds features: ", one_test_ds[0])
print("test ds: ", one_test_ds)

print(train_ds.feat_width())
print(test_ds.feat_width())

train ds len:  9296
train ds features:  tensor([1578463970544,          3853,          -999,          -999,
                 -999,          -999,          -999,          -999,
                 -999,          -999,          -999,          -999,
                 -999,          -999,           -86,           -85,
                  -86,          -999,          -999,          -999,
                  -86,          -999,          -999,           -86,
                 -999,          -999,          -999,          -999,
                 -999,          -999,          -999,          -999,
                 -999,          -999,          -999,          -999,
                 -999,          -999,          -999,          -999,
                 -999,          -999,          -999,          -999,
                 -999,          -999,          -999,          -999,
                 -999,          -999,          -999,          -999,
                 -999,          -999,          -999,          -999,
        

In [63]:
# Create Dataloader
train_dataloader = DataLoader(train_ds, batch_size=BATCH_SIZE)
test_dataloader = DataLoader(test_ds, batch_size=BATCH_SIZE)

In [93]:
class DLSTM(nn.Module):
    def __init__(self, lstm_input_size, lstm_hidden_size_1, lstm_hidden_size_2, lstm_num_layers, dense_neurons):
        super(DLSTM, self).__init__()
        self.lstm_input_size = lstm_input_size
        self.lstm_hidden_size_1 = lstm_hidden_size_1
        self.lstm_hidden_size_2 = lstm_hidden_size_2
        self.lstm_num_layers = lstm_num_layers
        self.dense_neurons = dense_neurons
        # LSTM: input of shape (seq_len, batch, input_size)
        self.lstm_1 = nn.LSTM(input_size=lstm_input_size, hidden_size=lstm_hidden_size_1, num_layers=lstm_num_layers)
        # self.lstm_2 = nn.LSTM(input_size=lstm_hidden_size_1, hidden_size=lstm_hidden_size_2)
        # self.batch_norm_1 = nn.BatchNorm1d(BATCH_SIZE)
        # self.sigmoid = nn.Sigmoid()
        self.dropout_1 = nn.Dropout(0.3)
        self.dense_1 = nn.Linear(lstm_hidden_size_1, dense_neurons)
        self.dropout_2 = nn.Dropout(0.3)
        self.dense_2 = nn.Linear(dense_neurons, 2)

    def forward(self, x, prints=False):
        # initialization
        # activation layer

        # # h0 shape should be: (num_layers, batch_size, hidden_size)
        h0 = torch.zeros(self.lstm_num_layers, x.size(1), self.lstm_hidden_size_1).to(DEVICE)
        c0 = torch.zeros(self.lstm_num_layers, x.size(1), self.lstm_hidden_size_1).to(DEVICE)
        if prints: print("hidden state temp shape: ", h0.shape)
        if prints: print("x before LSTM: ", x.shape)
        # x, _ = self.lstm_1(x)
        # x = self.batch_norm_1(x)
        x, _ = self.lstm_1(x, (h0, c0))
        x = F.relu(x)
        # x = self.sigmoid(x)
        # x, _ = self.lstm_2(x)
        # x = F.relu(x)
        # x = self.sigmoid(x)
        if prints: print("after LSTM + sigmoid: ", x.shape)

        x = self.dropout_1(x)
        x = F.relu(self.dense_1(x))
        if prints: print("after DO & Dense: ", x.shape)

        x = self.dropout_2(x)
        x = self.dense_2(x)
        return x

In [94]:
train_length = train_ds.__len__()
train_width = train_ds.feat_width()

model = DLSTM(train_width, 30, 40, 1, 60).to(DEVICE)
print(model)

# input_size = num of features, so length of columns
# sequence_length = 5

# Check if it works
train_batch_sample = next(iter(train_dataloader))
print("feature shape: ", train_batch_sample[0].shape)
print("x shape: ", train_batch_sample[1].shape)
print("y shape: ", train_batch_sample[2].shape)
train_batch_sample = train_batch_sample[0].unsqueeze(0)
print("input shape after reshaping: ", train_batch_sample.shape)
outputs = model(train_batch_sample.float(), prints=True)
print("final output: ", outputs)

DLSTM(
  (lstm_1): LSTM(197, 30)
  (dropout_1): Dropout(p=0.3, inplace=False)
  (dense_1): Linear(in_features=30, out_features=60, bias=True)
  (dropout_2): Dropout(p=0.3, inplace=False)
  (dense_2): Linear(in_features=60, out_features=2, bias=True)
)
feature shape:  torch.Size([4, 197])
x shape:  torch.Size([4])
y shape:  torch.Size([4])
input shape after reshaping:  torch.Size([1, 4, 197])
hidden state temp shape:  torch.Size([1, 4, 30])
x before LSTM:  torch.Size([1, 4, 197])
after LSTM + sigmoid:  torch.Size([1, 4, 30])
after DO & Dense:  torch.Size([1, 4, 60])
final output:  tensor([[[-0.0777, -0.0411],
         [-0.1220, -0.0797],
         [-0.1328, -0.0917],
         [-0.0192, -0.0242]]], grad_fn=<AddBackward0>)


In [95]:
train_width = train_ds.feat_width()
model = DLSTM(train_width, 30, 40, 1, 60).to(DEVICE)

In [96]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) # default lr: 5e-3

data_dict ={}
best_loss = 1000
best_epoch = 0
model.train()

for epoch in range(EPOCH):
    print("epoch: ", epoch)
    losses = []
    for i, data in enumerate(train_dataloader):
        features = data[0].to(DEVICE).float()
        features = features.unsqueeze(0)
        # print("imu shape: ", imu_features.shape)
        x = data[1].to(DEVICE).float().unsqueeze(-1)
        # print("x shape: ", x.shape)
        y = data[2].to(DEVICE).float().unsqueeze(-1)
        # print("y shape: ", y.shape)

        output = model(features, prints=False)
        # print("output", output)
        # print("output shape after concat: ", output.shape)
        label = torch.cat([x, y], dim=-1)
        # print("label", label)
        # print("label shape after concat: ", label.shape)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        print(f"loss:{np.mean(losses)} at iteration {i}")

print("Training finished")

epoch:  0
loss:13713.9404296875 at iteration 0
loss:13958.08154296875 at iteration 1
loss:14315.506510416666 at iteration 2
loss:14590.258056640625 at iteration 3
loss:14617.1814453125 at iteration 4
loss:14492.540201822916 at iteration 5
loss:14424.27162388393 at iteration 6
loss:14641.441528320312 at iteration 7
loss:14939.169596354166 at iteration 8
loss:15966.28759765625 at iteration 9
loss:17022.722389914772 at iteration 10
loss:17976.889729817707 at iteration 11
loss:18135.580603966348 at iteration 12
loss:18377.27197265625 at iteration 13
loss:18565.812825520832 at iteration 14
loss:18668.220275878906 at iteration 15
loss:18761.0322265625 at iteration 16
loss:18764.85812717014 at iteration 17
loss:18705.741724917763 at iteration 18
loss:18661.852685546874 at iteration 19
loss:18604.28976004464 at iteration 20
loss:18590.393510298294 at iteration 21
loss:18710.321713654892 at iteration 22
loss:19358.816284179688 at iteration 23
loss:20029.2931640625 at iteration 24
loss:20656.829

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:528: UserWarning: Using a target size (torch.Size([4, 2])) that is different to the input size (torch.Size([1, 4, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


ストリーミング出力は最後の 5000 行に切り捨てられました。
loss:2374.3235234369727 at iteration 462
loss:2371.6607960010397 at iteration 463
loss:2369.809351045342 at iteration 464
loss:2369.825988180136 at iteration 465
loss:2370.894138834461 at iteration 466
loss:2372.10219860892 at iteration 467
loss:2373.0200650148045 at iteration 468
loss:2373.3336360200924 at iteration 469
loss:2373.086625538561 at iteration 470
loss:2373.0650548773297 at iteration 471
loss:2374.4123408053442 at iteration 472
loss:2375.403070184249 at iteration 473
loss:2378.728738885177 at iteration 474
loss:2384.2189625331334 at iteration 475
loss:2389.0634399030196 at iteration 476
loss:2397.47916345317 at iteration 477
loss:2396.992373727309 at iteration 478
loss:2395.7326807022096 at iteration 479
loss:2391.8394230388553 at iteration 480
loss:2388.852689561013 at iteration 481
loss:2385.197976374972 at iteration 482
loss:2381.5501999658 at iteration 483
loss:2378.1033785161285 at iteration 484
loss:2375.1350540349513 at iteration 485


KeyboardInterrupt: ignored

In [ ]:
# def evaluate(model, data_loader,  device='cuda'):
#     model.to(device)
#     model.eval()
#     x_list = []
#     y_list = []
#     floor_list = []
#     prexs_list = []
#     preys_list = []
#     prefloors_list = []
#     for d in tqdm(data_loader):
#         data_dict['BSSID_FEATS'] = d['BSSID_FEATS'].to(device).long()
#         data_dict['RSSI_FEATS'] = d['RSSI_FEATS'].to(device).float()
#         data_dict['site_id'] = d['site_id'].to(device).long()
#         x = d['x'].to(device).float()
#         y = d['y'].to(device).float()
#         floor = d['floor'].to(device).long()
#         x_list.append(x.cpu().detach().numpy())
#         y_list.append(y.cpu().detach().numpy())
#         floor_list.append(floor.cpu().detach().numpy())
#         xy, floor = model(data_dict)
#         prexs_list.append(xy[:, 0].cpu().detach().numpy())
#         preys_list.append(xy[:, 1].cpu().detach().numpy())
#         prefloors_list.append(floor.squeeze().cpu().detach().numpy())
#     x = np.concatenate(x_list)
#     y = np.concatenate(y_list)
#     floor = np.concatenate(floor_list)
#     prexs = np.concatenate(prexs_list)
#     preys =np.concatenate(preys_list)
#     prefloors = np.concatenate(prefloors_list)
#     eval_score = comp_metric(x, y, floor, prexs, preys, prefloors)
#     return eval_score

# def get_result(model, data_loader, device='cuda'):
#     model.eval()
#     model.to(device)
#     prexs_list = []
#     preys_list = []
#     prefloors_list = []
#     data_dict = {}
#     for d in tqdm(data_loader):
#         data_dict['BSSID_FEATS'] = d['BSSID_FEATS'].to(device).long()
#         data_dict['RSSI_FEATS'] = d['RSSI_FEATS'].to(device).float()
#         data_dict['site_id'] = d['site_id'].to(device).long()
#         xy, floor = model(data_dict)
#         prexs_list.append(xy[:, 0].cpu().detach().numpy())
#         preys_list.append(xy[:, 1].cpu().detach().numpy())
#         prefloors_list.append(floor.squeeze(-1).cpu().detach().numpy())
#     prexs = np.concatenate(prexs_list)
#     preys =np.concatenate(preys_list)
#     prefloors = np.concatenate(prefloors_list)
#     return prexs, preys, prefloors